In [10]:
import pandas as pd
import numpy as np
import os

import groq 

tv_df=pd.read_csv("C:/Users/Srijan/Movie-recommender-application/src/data/Preprocessed/tv_preprocessed.csv")
movies_df=pd.read_csv("C:/Users/Srijan/Movie-recommender-application/src/data/Preprocessed/movies_preprocessed.csv")


def filter_by_genre(df, genres):
    return df[df['genre_names'].apply(lambda x: any(genre in x for genre in genres))]

def filter_by_popularity(df, threshold):
    return df[df['popularity'] >= threshold]

groq.api_key = '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'

def get_similar_overview(target_overview, candidates):
    prompt = f"Find the closest matching overviews to: {target_overview}\n\nCandidates:\n" + "\n".join(candidates)
    response = groq.Completion.create(
        model="llama3-8b-8192",
        prompt=prompt,
                max_tokens=1024
    )
    return response['choices'][0]['text'].strip().split('\n')


def recommend_movies(tv_series_title, user_preferences, top_n=5):
    # Layered filtering
    filtered_tv_df = tv_df[
        (tv_df['age_appropriate']) &
        (tv_df['language'] == user_preferences['language']) &
        (tv_df['popularity'] >= user_preferences['popularity_threshold'])
    ]
    filtered_tv_df = filter_by_genre(filtered_tv_df, user_preferences['genres'])    
    filtered_movies_df = movies_df[
        (movies_df['age_appropriate']) &
        (movies_df['language'] == user_preferences['language']) &
        (movies_df['popularity'] >= user_preferences['popularity_threshold'])
    ]
    filtered_movies_df = filter_by_genre(filtered_movies_df, user_preferences['genres'])
    
    # LLM-based similarity
    target_overview = tv_df[tv_df['name'] == tv_series_title]['overview'].values[0]
    movie_overviews = filtered_movies_df['overview'].tolist()
    similar_overviews = get_similar_overview(target_overview, movie_overviews)
    
    # Get top N recommendations
    recommendations = filtered_movies_df[filtered_movies_df['overview'].isin(similar_overviews)].head(top_n)
    return recommendations
